In [1]:
# Module Imports
import mariadb

# Execption used to stop the Notebook cell execution politely
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# Connect to the server and return a Connection object for the db_name

def connectToDB(db_name):
    try:
        return mariadb.connect(
        user="root",
        password="password",
        host="localhost", # Use of localhost because 1/ we don't have acces to the docker local network 2/ we have made a port redirection from themariadb server to the host
        port=3306,
        database=db_name
        )

    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        raise StopExecution
            
conn = connectToDB("flight_reservation")

Error connecting to MariaDB Platform: Access denied for user 'root'@'localhost' (using password: YES)


In [1]:
# A bit of dark magic? Just two intrincated query, one to get the TABLES, one for each TABLE to get the COLUMNS.
# And a clever use of format to have aligned columns.
def showTablesAndColumns(conn):
    cur = conn.cursor()
# Query the tables
    cur.execute("SHOW TABLES;")
# For each table
    for table in cur.fetchall():
        print("**",table[0],"**")
# Defining the output format {:16} indicate at least 16 characters, add spaces if necessary
        outputFormat="{:16}\t{:10}\t{}\t{}\t{}\t{}"
# Printing the headers
        print(outputFormat.format("Field","Type","Null","Key","Default","Extra"))
# Query the columns
        cur.execute('SHOW COLUMNS FROM {};'.format(table[0]))
# For each column
        for attribute in cur.fetchall(): # Print the attribute (*attribute give the content of the tuple to the format function)
            print(outputFormat.format(*attribute))
            
showTablesAndColumns(conn)


NameError: name 'conn' is not defined

In [ ]:
import time

def addFlight(id_flight, maximum_capacity):
    try:
        cur = conn.cursor()
        # Insert a new flight with current_capacity set to 0
        cur.execute(
            "INSERT INTO Flight (id_flight, maximum_capacity, current_capacity) VALUES (?, ?, ?);",
            (id_flight, maximum_capacity, 0)
        )
        conn.commit()
        print(f"Flight with id {id_flight} added successfully.")
    except mariadb.Error as e:
        print(f"Error adding flight: {e}")


def addBooking(id_flight, client_name, quantity):
    try:
        cur = conn.cursor()
        # Check if the flight exists
        cur.execute("SELECT maximum_capacity, current_capacity FROM Flight WHERE id_flight=?;", (id_flight,))
        flight = cur.fetchone()
        print(f"{client_name} flight checked")
        if not flight:
            return (False, "Invalid")
        maximum_capacity, current_capacity = flight
        # Check if there are enough available seats
        if current_capacity + quantity > maximum_capacity:
            return (False, "Full")
        # Simulate payment processing
        time.sleep(3)
        # Insert a new booking
        cur.execute(
            "INSERT INTO Booking (id_flight, client_name, quantity) VALUES (?, ?, ?);",
            (id_flight, client_name, quantity)
        )
        print(f"{client_name} insterted")
        # Update the flight's current capacity
        cur.execute(
            "UPDATE Flight SET current_capacity = current_capacity + ? WHERE id_flight = ?;",
            (quantity, id_flight)
        )
        print(f"{client_name} flight updated")
        conn.commit()
        print(f"{client_name} commited")
        return (False, "Reserved")
    except mariadb.Error as e:
        print(f"Error adding booking {client_name}: {e}")
        return (True, "Error")

In [ ]:
def cleanDB(id_flight):
    conn = connectToDB("flight_reservation")
    cur = conn.cursor()
    cur.execute("DELETE FROM Booking WHERE id_flight=?;",(id_flight,))
    cur.execute("UPDATE Flight SET current_capacity=0 WHERE id_flight=?;",(id_flight,))
    conn.commit() # Without commit the modification are not applied to the database

In [ ]:
def processBooking(id_flight, client_name, quantity):
    print("Processing Flight",id_flight,"for",client_name,":", quantity,"seats.")
    retry,status = addBooking(id_flight,client_name,quantity)
    print(client_name,":",status, "Should retry:",retry)

In [ ]:
cleanDB("LH6795")

In [5]:
import threading
import random
import time

# Test parameters
id_flight="LH6795"
base_name="Client "
# Clean the db for the test
cleanDB(id_flight)
# Creating several Thread representing several client that try to book simultaneously a random number of seat
jobs=[]
for i in range(5):
    jobs.append(threading.Thread(target=processBooking,args=(id_flight,base_name+str(i), random.randint(1,4))))
# Start each job, giving 3s of delay between each
for job in jobs:
    job.start()
    time.sleep(3)
# Wait for all the thread to finish
for job in jobs:
    job.join()
print("Finished")

Exception in thread Thread-3 (processBooking):
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\miniconda3\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 64, in processBooking
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 27, in addBooking
TypeError: cannot unpack non-iterable bool object


Processing Flight LH6795 for Client 0 : 1 seats.
Client 0 flight checked


Exception in thread Thread-4 (processBooking):
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\miniconda3\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 64, in processBooking
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 27, in addBooking
TypeError: cannot unpack non-iterable bool object


Processing Flight LH6795 for Client 1 : 4 seats.
Client 1 flight checked


Exception in thread Thread-5 (processBooking):
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\miniconda3\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 64, in processBooking
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 27, in addBooking
TypeError: cannot unpack non-iterable bool object


Processing Flight LH6795 for Client 2 : 1 seats.
Client 2 flight checked


Exception in thread Thread-6 (processBooking):
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\miniconda3\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 64, in processBooking
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 27, in addBooking
TypeError: cannot unpack non-iterable bool object


Processing Flight LH6795 for Client 3 : 1 seats.
Client 3 flight checked


Exception in thread Thread-7 (processBooking):
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\miniconda3\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\ProgramData\miniconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 64, in processBooking
  File "C:\Users\mrnew\AppData\Local\Temp\ipykernel_15264\1715268380.py", line 27, in addBooking
TypeError: cannot unpack non-iterable bool object


Processing Flight LH6795 for Client 4 : 1 seats.
Client 4 flight checked
Finished
